In [1]:
import pandas as pd

In [2]:
df_amneties= pd.read_csv("final/final_building_amenities.csv")
df_amneties.head(2)

,building_name,area_name,latitude,longitude,transport,metro_stations,gyms,spas,supermarkets,malls,schools,restaurant,hospital,tourist_attraction,park,worship
0,"Delphine, Dubai Marina",Dubai Marina,25.073213,55.132712,"[{'business_status': 'OPERATIONAL', 'name': 'R...","[{'business_status': 'OPERATIONAL', 'name': 'D...","[{'business_status': 'OPERATIONAL', 'name': 'T...","[{'business_status': 'OPERATIONAL', 'name': 'C...","[{'business_status': 'OPERATIONAL', 'name': 'W...","[{'business_status': 'OPERATIONAL', 'name': 'I...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'L...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'D...","[{'business_status': 'OPERATIONAL', 'name': 'S...","[{'business_status': 'CLOSED_TEMPORARILY', 'na..."
1,"Marina Plaza, Dubai Marina",Dubai Marina,25.075398,55.139825,"[{'business_status': 'OPERATIONAL', 'name': 'R...","[{'business_status': 'OPERATIONAL', 'name': 'D...","[{'business_status': 'OPERATIONAL', 'name': 'T...","[{'business_status': 'OPERATIONAL', 'name': 'C...","[{'business_status': 'OPERATIONAL', 'name': 'W...","[{'business_status': 'OPERATIONAL', 'name': 'I...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'L...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'Q...","[{'business_status': 'OPERATIONAL', 'name': 'S...","[{'business_status': 'CLOSED_TEMPORARILY', 'na..."


In [5]:
print(df_amneties.loc[0])

building_name                                    Delphine, Dubai Marina
area_name                                                  Dubai Marina
latitude                                                      25.073213
longitude                                                     55.132712
transport             [{'business_status': 'OPERATIONAL', 'name': 'R...
metro_stations        [{'business_status': 'OPERATIONAL', 'name': 'D...
gyms                  [{'business_status': 'OPERATIONAL', 'name': 'T...
spas                  [{'business_status': 'OPERATIONAL', 'name': 'C...
supermarkets          [{'business_status': 'OPERATIONAL', 'name': 'W...
malls                 [{'business_status': 'OPERATIONAL', 'name': 'I...
schools               [{'business_status': 'OPERATIONAL', 'name': 'A...
restaurant            [{'business_status': 'OPERATIONAL', 'name': 'L...
hospital              [{'business_status': 'OPERATIONAL', 'name': 'A...
tourist_attraction    [{'business_status': 'OPERATIONAL', 'name'

In [ ]:
def create_knowledge_graph(self, df_amenities: pd.DataFrame):
    with self.driver.session() as session:
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (a:Area) REQUIRE a.name IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (b:Building) REQUIRE b.name IS UNIQUE")
        # ... existing constraints for other nodes ...

        # First, create all areas
        areas = df_amenities['area_name'].unique()
        for area_name in areas:
            self._create_area(session, area_name)

        # Then create buildings and their relationships
        for _, row in df_amenities.iterrows():
            self._create_building_with_amenities(session, row)

def _create_area(self, session, area_name: str):
    query = """
    MERGE (a:Area {name: $area_name})
    """
    session.run(query, area_name=area_name)

def _create_building_with_amenities(self, session, row):
    # First ensure building is connected to area
    query = """
    MATCH (a:Area {name: $area_name})
    MERGE (b:Building {name: $building_name})
    SET b.latitude = $lat, b.longitude = $lng
    MERGE (a)-[:CONTAINS]->(b)
    """
    session.run(
        query,
        area_name=row['area_name'],
        building_name=row['building_name'],
        lat=row['latitude'],
        lng=row['longitude']
    )

    # Then create amenity relationships
    self._create_amenity_relationships(
        session, 
        row['building_name'],
        'metro_stations',
        row['metro_stations'],
        'MetroStation',
        'HAS_METRO'
    )
    
    self._create_amenity_relationships(
        session,
        row['building_name'],
        'hospitals',
        row['hospital'],
        'Hospital',
        'HAS_HOSPITAL'
    )
    # ... similar for other amenities ...

def _create_amenity_relationships(
    self, session, building_name: str, 
    amenity_type: str, amenities: List[Dict],
    node_label: str, relationship_type: str
):
    for amenity in amenities:
        query = f"""
        MATCH (b:Building {{name: $building_name}})
        MERGE (am:{node_label} {{name: $amenity_name, vicinity: $vicinity}})
        SET am.business_status = $status
        MERGE (b)-[:{relationship_type}]->(am)
        """
        session.run(
            query,
            building_name=building_name,
            amenity_name=amenity['name'],
            vicinity=amenity['vicinity'],
            status=amenity['business_status']
        )

In [ ]:
class RealEstateRAG:
    def __init__(self, kg: RealEstateKnowledgeGraph):
        self.kg = kg
        
    def generate_building_recommendation(self, user_query: str):
        """
        Generate recommendations based on user query
        """
        # Extract preferences from user query using LLM
        school_priority = self._extract_school_priority(user_query)
        
        # Query knowledge graph
        recommendations = self.kg.query_building_recommendations(
            school_priority=school_priority
        )
        
        # Generate natural language response
        response = self._generate_response(recommendations, user_query)
        return response

    def _extract_school_priority(self, query: str) -> float:
        # Use LLM to analyze query and return school priority
        # Example implementation:
        if "school" in query.lower():
            return 0.7
        return 0.5

    def _generate_response(self, recommendations: List, query: str) -> str:
        # Use LLM to generate natural language response
        # Example implementation:
        response = "Based on your requirements, here are the top recommendations:\n\n"
        for building, score in recommendations:
            response += f"- {building} (Score: {score:.2f})\n"
        return response